# DS Phase: Load shared data, process, publish result, and verify save/load

In [ ]:
import beaver

session = beaver.active_session()
assert session is not None, 'Session should be available'

# Try to resume if we already processed
try:
    session.load(overwrite=True)
    if 'phase' in globals() and phase == 'processed':
        print('Already processed; skipping work')
        raise SystemExit(0)
except FileNotFoundError:
    pass

shared_data = session.wait_for_remote_var('shared_data', auto_accept=True, trust_loader=True)
# Ensure we have the value, not just a pointer
if hasattr(shared_data, 'load'):
    shared_data = shared_data.load(auto_accept=True, trust_loader=True)
elif hasattr(shared_data, 'value'):
    shared_data = shared_data.value
print('Got shared data:', shared_data)

processed_data = shared_data * 2
session.remote_vars['processed_data'] = processed_data

phase = 'processed'
state_path = session.save()
print(f'State saved to: {state_path}')

# Simulate restart by clearing globals then reloading
shared_data = None
processed_data = None
phase = None
session.load(path=state_path, overwrite=True)
assert processed_data == shared_data * 2
print('State restored and verified after restart')
